## Building makemore Part 3: Activations & Gradients, BatchNorm
#### https://www.youtube.com/watch?v=P6sfmUTpUmc

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
words = open("data/names.txt", "r").read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [5]:
chars = sorted(list(set("".join(words))))
stoi = {s:i for i,s in enumerate(chars,1)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [8]:
block_size = 3
X, y = [], []

random.seed(42)
random.shuffle(words)

for name in words:
    window = [0] * block_size
    for n in name + ".":
        n = stoi[n]
        X.append(window)
        y.append(n)
        window = window[1:] + [n]

X = torch.tensor(X)
y = torch.tensor(y)

print(X.shape)
print(y.shape)

t = int(0.8*len(X))
d = int(0.9*len(X))
X_train, y_train = X[:t], y[:t]
X_val, y_val = X[t:d], y[t:d]
X_test, y_test = X[d:], y[d:]

torch.Size([228146, 3])
torch.Size([228146])


In [9]:
print(len(X_train), len(y_train))
print(len(X_val), len(y_val))
print(len(X_test), len(y_test))

182516 182516
22815 22815
22815 22815


In [10]:
v_size = 10
h_size = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,v_size), generator=g)
W1 = torch.randn((block_size * v_size, h_size), generator=g)
b1 = torch.randn(h_size, generator=g)
W2 = torch.randn((h_size,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [11]:
for p in parameters:
    p.requires_grad = True

In [ ]:
batch_size = 32
lr = 0.1

for _ in range(10000):

    ix = torch.randint(0, X_train.shape[0], (batch_size,))
    
    # Forward pass
    emb = C[X_train[ix]]
    h = torch.tanh((emb.view(-1,block_size * v_size) @ W1) + b1)
    logits = (h @ W2) + b2
    loss = F.cross_entropy(logits, y_train[ix])
    
    # Backwards pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update
    for p in parameters:
        p.data += -lr * p.grad

In [18]:
@torch.no_grad()
def check_loss(batch:str):
    batches = {"train":(X_train,y_train),
               "val":(X_val,y_val),
               "test":(X_test, y_test)}
    X = batches[batch][0]
    y = batches[batch][1]
    emb = C[X]
    h = torch.tanh((emb.view(-1,block_size * v_size) @ W1) + b1)
    logits = (h @ W2) + b2
    loss = F.cross_entropy(logits, y)
    
    print(f"{batch} loss: {loss}")

In [19]:
check_loss("val")

val loss: 26.08016586303711
